# Configure
Set `jvm_path` to your java virtual machine full path

In [ ]:
jvm_path = None

Set up logging

In [ ]:
import logging
logging.basicConfig(
  level=logging.INFO,
  format='%(asctime)s %(name)-12s %(levelname)-8s: %(message)s',
  datefmt='%Y-%m-%d %H:%M:%S',
)

Install package

In [ ]:
import sys
import os

cmd = "{} -m pip install -U .. --use-feature=in-tree-build".format(
  sys.executable
)
os.system(cmd);

Install dependencies

In [ ]:
cmd = "{} -m pip install -Ur ../notebooks-requirements.txt".format(
  sys.executable
)
os.system(cmd);

Start jvm

In [ ]:
from featgraph.jwebgraph import start_jvm
start_jvm(jvm_path=jvm_path)

Import the java library

In [ ]:
from it.unimi.dsi import webgraph

# Hands on: a toy example to become acquainted with the libraries
## Load graph
Let's load the following graph

![graph image](https://doc-0c-ag-docs.googleusercontent.com/docs/securesc/c275d3us2tonk8js4gjnts3eu7icvfro/5ro7ls5d0hfeosatrli98dr8m29pg2th/1626095325000/10295162260986597706/10295162260986597706/1t-EIZN1QCEou6Igv9s6dV8etah9UIRTP?e=download&authuser=0&nonce=ld6blsnpmc37e&user=10295162260986597706&hash=9g6g41jqk4tvsmrq4ra6sfp8jea9se7h)

In [ ]:
ascii_graph_str = """0 1
0 4
1 2
2 1
2 3
3 2
3 5
4 0
4 1
5 4"""

example_file = "graphs/example.arcs"
os.makedirs(os.path.dirname(example_file), exist_ok=True)
with open(example_file, "w") as f:
  f.write(ascii_graph_str)

In [ ]:
graph = webgraph.ArcListASCIIGraph.load(example_file)
print(graph)

## Convert
Convert graph to BVGraph

In [ ]:
bv_file_prefix = ".".join((*example_file.split(".")[:-1], "bvgraph"))
webgraph.BVGraph.store(graph, bv_file_prefix)

Print contents of properties file

In [ ]:
with open(".".join((bv_file_prefix, "properties"))) as f:
  print(f.read())

## Convert back

In [ ]:
reconvert_fname = ".".join((
  *example_file.split(".")[:-1],
  "reconverted", "arcs"
))
bvgraph = webgraph.BVGraph.load(bv_file_prefix)
webgraph.ArcListASCIIGraph.store(bvgraph, reconvert_fname)

Print `ArcListASCIIGraph` file

In [ ]:
with open(reconvert_fname) as f:
  print(f.read())

## Graph transformation

In [ ]:
print(webgraph.Transform.transpose(graph))

In [ ]:
graph_s = webgraph.Transform.symmetrize(graph)
graph_s_str = graph_s.toString().replace(
  "Arcs: unknown",
  "Arcs: {}".format(sum(map(
    graph_s.outdegree,
    graph_s.nodeIterator()
  )))
)
print(graph_s_str)

## Stats

In [ ]:
from it.unimi import dsi
import jpype.types

graph_bv = webgraph.BVGraph.load(bv_file_prefix)
buckets = jpype.types.JObject(None, dsi.bits.LongArrayBitVector)
sccsize = jpype.types.JObject(None, jpype.types.JArray(jpype.types.JInt))
stats_prefix = ".".join((*bv_file_prefix.split("."), "stats"))
pl = dsi.logging.ProgressLogger()

webgraph.Stats.run(graph_bv, buckets, sccsize, stats_prefix, pl)

Print out stats

In [ ]:
import glob

for i, fname in enumerate(glob.glob(stats_prefix + "*")):
  if i:
    print("-" * 40, "\n")
  print(os.path.basename(fname), "\n")
  with open(fname) as f:
    print(f.read())

## Degree correlation

In [ ]:
from it.unimi.dsi import law

def get_degrees(g):
  return jpype.types.JArray(jpype.JDouble)(
    list(map(g.outdegree, g.nodeIterator()))
  )
out_degrees = get_degrees(graph)
in_degrees = get_degrees(webgraph.Transform.transpose(graph))

print("KendallTau:", law.stat.KendallTau.INSTANCE.compute(
  out_degrees, in_degrees
))

Degree scatterplot

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

np.random.seed(2)
scatter_data = np.array([
  out_degrees, in_degrees
]) + np.random.randn(2, graph.numNodes()) * 0.01

plt.scatter(*scatter_data, c="k", marker="1")
plt.xticks(np.arange(2) + 1)
plt.yticks(np.arange(3) + 1)
plt.xlabel("out-degree")
plt.ylabel("in-degree")
plt.gca().set_aspect("equal", adjustable="box");